# Notebook for development and testing

In [1]:
def init_train_dataloader():
    """
        Init of the train data loader. NOT TESTED FOR MULTIPLE GPU
        Creating wrapper arround data class. 
    """
    _train_ds = ship_dataset("/home/franko/Desktop/DSelimovcProject/Dataset/", 'train')
    batch_size = 32
    if False and False:
        batch_size *= torch.cuda.device_count()

    _train_dl = DataLoader(
        _train_ds,
        batch_size=batch_size,
        num_workers=1,
        pin_memory=False,
    )  
    return _train_dl

In [3]:
from torch.utils.data import DataLoader
from DataLoader import ship_dataset
from Models import CNN_REG

dl = init_train_dataloader()

for batch in dl:
    _input, _output, _ = batch
    break
model = CNN_REG(max_pooling_ratio = 4)
model = model.double()
print(_input.shape)
pred = model(_input)
print(pred.shape)


torch.Size([32, 1, 3, 1501])
torch.Size([32, 3])
